In [1]:
# 导入需要的包
import numpy as np 
import pandas as pd
import math
import os
import re,string


#### 读出诗名

In [2]:
titles=os.listdir('dataset')

#### 读出作者和内容

In [3]:
doc_paths=[os.path.join('dataset',title)for title in titles]
authors=[]
contexts=[]
for i in range(len(doc_paths)):
    with open(doc_paths[i]) as f:
        # for line in f.readlines:
        authors.append(f.readline())
        contexts.append(f.read())


In [4]:
for i in range(len(titles)):
    titles[i]=titles[i].strip('.txt')
titles = [x.lower() for x in titles]
for i in range(len(authors)):
    authors[i]=authors[i].strip('\n')
    authors[i]=authors[i].strip('Author: ')
authors = [x.lower() for x in authors]
remove_punct_map = dict.fromkeys(map(ord, re.sub('\'','',string.punctuation)))
for i in range(len(contexts)):
    contexts[i]=contexts[i].replace('\n',' ')
    new_str=""
    for word in contexts[i].split(' '):
        new_str+=word.translate(remove_punct_map)
        new_str=new_str+' '
    contexts[i]=new_str
        
contexts = [x.lower() for x in contexts]

#### 对于这三个作用域建立词袋 

In [5]:
# 改变大小写，并进行排序
def sort_str(wordSet):
    listtemp = [x.lower() for x in wordSet]
    # 对元组排序，因为元组为：（忽略大小写的字符串，字符串），就是按忽略大小写的字符串排序
    listtemp.sort()   
    return listtemp

In [6]:
# 作者词袋
# 一个作者看成一个整体，对准确性要求太高
# author_wordset=set(authors)
# author_wordset=sort_str_author(author_wordset)
author_wordset=set(authors[0].split(' '))
for i in range(len(authors)-1):
    author_wordset=set(author_wordset).union(authors[i+1].split(' '))
# print(title_wordset)
author_wordset=sort_str(author_wordset)

# 标题词袋
title_wordset=set(titles[0].split(' '))
for i in range(len(titles)-1):
    title_wordset=set(title_wordset).union(titles[i+1].split(' '))
# print(title_wordset)
title_wordset=sort_str(title_wordset)
# print(title_wordset)

# 正文词袋
text_wordset=set(contexts[0].split(' '))
for i in range(len(contexts)-1):
    text_wordset=set(text_wordset).union(contexts[i+1].split(' '))
# new_set=set()
# # 去掉标点    可以考虑一下‘s，这个不删
# for text in text_wordset:
#     remove_punct_map = dict.fromkeys(map(ord, re.sub('\'','',string.punctuation)))
#     new_set.add(text.translate(remove_punct_map))
# text_wordset=new_set
# text_wordset.remove('')
text_wordset=sort_str(text_wordset)



In [7]:
#2.统计词项tj在文档Di中出现的次数，也就是词频。
def computeTF(wordSet,area_list):
    tf=[]
    for i in range(len(area_list)):
        tf.append(dict.fromkeys(wordSet, 0))
    # print(tf[0])
    for i in range(len(area_list)):
        split=area_list[i].split(' ')
        for word in split:
            if word in wordSet:
                tf[i][word] += 1
    return tf


In [8]:
#3.计算逆文档频率IDF
def computeIDF(tfList): 
    idfDict = dict.fromkeys(tfList[0],0) #词为key，初始值为0
    # print("1",idfDict)
    N = len(tfList)  #总文档数量 
    for tf in tfList: # 遍历字典中每一篇文章
        for word, count in tf.items(): #遍历当前文章的每一个词
            if count > 0 : #当前遍历的词语在当前遍历到的文章中出现
                idfDict[word] += 1 #包含词项tj的文档的篇数df+1  
    for word, Ni in idfDict.items(): #利用公式将df替换为逆文档频率idf
        idfDict[word] = math.log10(N/Ni)  #N,Ni均不会为0
    return idfDict   #返回逆文档频率IDF字典

In [9]:
#4.计算tf-idf(term frequency–inverse document frequency)
def computeTFIDF(tflist, idfs): #tf词频,idf逆文档频率   
    tfidf_list = []
    for i in range(len(tflist)):
        tf=tflist[i]
        tfidf_list.append({})
        for word, tfval in tf.items():
            tfidf_list[i][word] = tfval * idfs[word]
    return tfidf_list

In [10]:
tf_title=computeTF(title_wordset,titles)
tf_author=computeTF(author_wordset,authors)
tf_text=computeTF(text_wordset,contexts)
idfs_title=computeIDF(tf_title)
idfs_author=computeIDF(tf_author)
idfs_text=computeIDF(tf_text)
tfidf_title=computeTFIDF(tf_title,idfs_title)
tfidf_author=computeTFIDF(tf_author,idfs_author)
tfidf_text=computeTFIDF(tf_text,idfs_text)



In [11]:
def cos_same(query,idf,wordset,tfidf_list):
    result=[]
    if query=="":
        for i in range(len(tfidf_list)):
            result.append(0)
    query=query.strip('\n')
    new_str=""
    for word in query.split(' '):
        word = word.lower() 
        new_str+=word.translate(remove_punct_map)
        new_str=new_str+' '
    # 去掉最后一个空格
    new_str=new_str.strip(' ')
    query=new_str
    query_list=[]
    query_list.append(query)
    query=query_list
    # print(query)
    tf_q=computeTF(wordset,query)
    tfidf_q=computeTFIDF(tf_q,idf)
    # print(tfidf_q)
    # 计算余弦相似度，不要分母
    for i in range(len(tfidf_list)):
        temp=0
        # print(i,type(tfidf_list[i]))
        for word in tfidf_list[i].keys():
            temp+=tfidf_list[i][word]*tfidf_q[0][word]
        result.append(temp)
        # print("end")

    return result


    

In [12]:
# q类型为“author:A,title:B,”
# q=input("qingshuru1")
# q="Rabindranath Tagore"
# cos_same(q,idfs_author,author_wordset,tfidf_author)
query_title=input("诗名")
query_author=input("作者")
query_text=input("正文")
title_result=cos_same(query_title,idfs_title,title_wordset,tfidf_title)
author_result=cos_same(query_author,idfs_author,author_wordset,tfidf_author)
text_result=cos_same(query_text,idfs_text,text_wordset,tfidf_text)
result=[]
for i in range(len(titles)):
    result.append(title_result[i]+author_result[i]+text_result[i])
zero_id=[]
for i in range(len(result)):
    if result[i]==0:
        zero_id.append(i)

sorted_id = sorted(range(len(result)), key=lambda k: result[k], reverse=True)
for i in sorted_id:
    if i not in zero_id:
        print(titles[i])




freedom
tonight i can write
where the mind is without fear
aedh wishes for the cloths of heaven
down by the salley gardens
when you are old
walk with me in moonligh
a drinking song
leave this
